# Debug notebook
This notebook is only used to troubleshoot the S2 scene workflow processor.

In [1]:
import os
import glob
import shutil
from datetime import datetime
import pandas as pd
import time

from utils.prepS2 import *
from utils.yamlUtils import *

In [2]:
non_cogs_dir = "/data/S2_Download/"
cogs_dir = "/data/S2_ARD/"
sen2cor_8 = "/Sen2Cor-02.08.00-Linux64/bin/L2A_Process"
sen2cor_5 = "/Sen2Cor-02.08.00-Linux64/bin/L2A_Process"

s3_bucket = "public-eo-data"
s3_dir = "catapult/sentinel2/"

access_key = os.getenv("AWS_ACCESS_KEY")
secret_key = os.getenv("AWS_SECRET_KEY")

In [3]:
os.makedirs(non_cogs_dir, exist_ok=True)
os.makedirs(cogs_dir, exist_ok=True)

In [4]:
def process_scene(des_scene):    
    # shorten scene name
    scene_name = des_scene[:-21]
    scene_name = scene_name[:-17] + scene_name.split('_')[-1]
    print ( 'Scene name: {}'.format(scene_name) )

    # find uuid for download via esa hub
    s2id = find_s2_uuid(des_scene)

    down_dir = non_cogs_dir + des_scene + '/'
    print ( 'Download scene dir: {}'.format(down_dir) )
    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )
    #download_s2_granule_gcloud(des_scene, non_cogs_dir)
    download_extract_s2_esa(s2id, non_cogs_dir, down_dir) # using esa due to gcloud metadata issues + sedas confidence

    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

    # process to l2a
    l2a_dir = down_dir.replace('_MSIL1C', '_MSIL2A')
    print ( 'L2A temp dir: {}'.format(l2a_dir) )
    if not os.path.exists(l2a_dir):
        sen2cor_correction(sen2cor_8, down_dir, non_cogs_dir) # May need to include try of version 2.5 before certain date - to be determined...

    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

    # convert to cog
    cog_dir = cogs_dir + scene_name.replace('_MSIL1C', '_MSIL2A') + '/'
    print ( 'COG dir: {}'.format(cog_dir) )
    conv_s2scene_cogs(l2a_dir, cog_dir, scene_name.replace('_MSIL1C', '_MSIL2A'))

    copy_s2_metadata(l2a_dir, cog_dir, scene_name.replace('_MSIL1C', '_MSIL2A'))

    create_yaml(cog_dir, 's2')

    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

In [5]:
import rediswq

host="redis"
# Uncomment next two lines if you do not have Kube-DNS working.
# import os
# host = os.getenv("REDIS_SERVICE_HOST")

q = rediswq.RedisWQ(name="job2", host=host)
print("Worker with sessionID: " +  q.sessionID())
print("Initial queue state: empty=" + str(q.empty()))

Worker with sessionID: 2cc7106e-3651-49e0-b7da-7d14738f5e9a
Initial queue state: empty=False


In [6]:
while not q.empty():
    item = q.lease(lease_secs=10, block=True, timeout=2) 
    if item is not None:
        itemstr = item.decode("utf=8")
        print("Working on " + itemstr)
        #time.sleep(10) # Put your actual work here instead of sleep.
        process_scene(itemstr)
        q.complete(item)
    else:
        print("Waiting for work")

Working on S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE
Scene name: S2A_MSIL1C_20180820T223011_T60KWE
Download scene dir: ./S2_Download/S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE/
Time: 2019-08-30 07:37:06


Downloading: 100%|██████████| 589M/589M [13:09<00:00, 711kB/s] 
MD5 checksumming: 100%|██████████| 589M/589M [00:02<00:00, 258MB/s] 


Extracting ESA scene: ./S2_Download/S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE/
Deleting ESA scene zip: ./S2_Download/S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.zip
Time: 2019-08-30 07:50:21
L2A temp dir: ./S2_Download/S2A_MSIL2A_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE/
/Sen2Cor-02.08.00-Linux64/bin/L2A_Process --output_dir ./S2_Download/ ./S2_Download/S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE/
Time: 2019-08-30 08:13:59
COG dir: ./S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/
Creating scene cog directory: ./S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/
./S2_Download/S2A_MSIL2A_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE/GRANULE/L2A_T60KWE_A016514_20180820T223008/IMG_DATA/R60m/T60KWE_20180820T223011_B09_60m.jp2 ./S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2A_20180820T223011_T60KWE_B09_60m.tif
./S2_Download/S2A_MSIL2A_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE/GRANULE/L2A_T60KWE_A016514_2

In [7]:
print("Queue empty!")

Queue empty!
